In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
from PIL import Image
from dataset import *
from save_load import *
from NN_library.AutoUnet.AutoUnet_uniform import *
from NN_library.AutoUnet.train_AutoUnet import *
%load_ext autoreload
%autoreload 2

Using device: cuda


In [2]:
data = dataset_uniform()
geometry, damage, imp_shrinkage, obs_shrinkage, stiffness = data.__getitem__(1000)
loaders = get_loaders(data, 16)

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=11, figsize=(11, 3), dpi=200)
for i in range(10):
    axs[0,i+1].imshow(geometry[0], cmap='Greys', vmin=0, vmax=1)
    axs[0,i+1].set_title(f't = {i+1}')
    axs[0,i+1].set_axis_off()
    axs[1,i+1].imshow(imp_shrinkage[i+1] / (-0.001), cmap='Greys', vmin=0, vmax=1)
    axs[1,i+1].set_axis_off()
    axs[2,i+1].imshow(damage[i], cmap='Greys')
    axs[2,i+1].set_axis_off()
axs[0,0].text(0.8, 0.05, 'geometry', rotation='vertical')
axs[0,0].set_axis_off()
axs[1,0].text(0.8, 0.35, 'load', rotation='vertical')
axs[1,0].set_axis_off()
axs[2,0].text(0.8, 0.2, 'damage', rotation='vertical')
axs[2,0].set_axis_off()

In [10]:
name = 'NN_library/AutoUnet/AutoUnet_uniform'
args = {'lr' : 0.00001, 'epochs' : 50, 'dev' : dev, 'name' : name}

In [4]:
unet = AutoUNet()

In [5]:
torch.cuda.empty_cache() 

In [6]:
total_params = sum(p.numel() for p in unet.parameters())
print(f"Number of parameters: {total_params}")

Number of parameters: 7697345


In [20]:
unet = load_network(unet, args['name']+'_49', args)

In [21]:
losses_train, losses_val = train(unet, loaders, args)

Epoch: 0 batch: 0 mean train loss:  840.1463798636
Epoch: 0 batch: 100 mean train loss:  1002.6577028659
Epoch: 0 batch: 200 mean train loss:  577.0496532462
Epoch: 0 batch: 300 mean train loss:  568.9809247187
Epoch: 0 batch: 400 mean train loss:  511.2796282112
Epoch: 0 batch: 500 mean train loss:  693.5641025315
Epoch: 0 batch: 600 mean train loss:  547.0508371428
Epoch: 0 batch: 700 mean train loss:  740.8246180992
Epoch: 0 mean train loss:  727.9608611223 mean val. rec. loss:  912.9660644531
Epoch: 1 batch: 0 mean train loss:  957.3009239585
Epoch: 1 batch: 100 mean train loss:  683.8215237385
Epoch: 1 batch: 200 mean train loss:  533.6452658303
Epoch: 1 batch: 300 mean train loss:  587.0199815284
Epoch: 1 batch: 400 mean train loss:  601.0291978614
Epoch: 1 batch: 500 mean train loss:  518.6598832233
Epoch: 1 batch: 600 mean train loss:  680.3625043667
Epoch: 1 batch: 700 mean train loss:  870.8016365932
Epoch: 1 mean train loss:  724.5163178877 mean val. rec. loss:  910.58642578

In [19]:
plt.plot(losses_train)
plt.plot(losses_val)
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

## Train with Convnet

In [ ]:
from NN_library.ConvNet.ConvNet_uniform import *

name = 'NN_library/AutoUnet/AutoUnet_w_Conv_uniform'
conv_name = 'NN_library/ConvNet/Conv_w_AutoUnet_uniform'

In [ ]:
args = {'lr' : 0.000001, 'epochs' : 20, 'dev' : dev, 'name' : name, 'conv_name' : conv_name}

In [ ]:
unet = AutoUNet()
convnet = ConvNet(16)

In [ ]:
convnet = load_network(convnet, 'NN_library/ConvNet/ConvNet_uniform', args)

In [ ]:
unet = load_network(unet, 'NN_library/AutoUnet/AutoUnet_uniform', args)  

In [ ]:
losses_train, losses_val = train_w_Conv(unet, convnet, loaders, args)

In [ ]:
plt.plot(losses_train)
plt.plot(losses_val)
plt.xlabel('epoch')
plt.legend(['train', 'validation'])